In [100]:
# Set up imports
import sys
sys.path.append('/Users/diegodavila/miniconda3/lib/python3.7/site-packages')
import flywheel
import nibabel as nb
import os
import re
import numpy as np 
import pandas as pd 

# set up flywheel instance
fw = flywheel.Client()
projectContainer = fw.projects.find("label=GRMPY_822831")
projectContainer = projectContainer[0]
sessions = projectContainer.sessions()
subjects = projectContainer.subjects()

#get list of IDs who are affected
recon_id = pd.read_csv("grmpy_aslrecon.csv", dtype=str)
done_id = pd.read_csv("already_done.csv", dtype=str)

## Set up identifier functions

In [44]:
def identify_recon(ses):
    asl_to_return = None
    is_asl = False
    for acq in ses.acquisitions():
        filelist = [x for x in acq.files if x.type in ["nifti"]]
        for f in filelist:
            #print("searching for {}_.+x{} in {}".format(ses.subject.label,ses.label, f.name))
            search = re.search(r"{}_.+x{}".format(ses.subject.label,ses.label), f.name)
            if search is not None:
                asl_to_return = f
                is_asl = True
            else:
                continue
        if is_asl:
            return asl_to_return

In [45]:
def recon_index(ses):
    asl_to_return = None
    is_asl = False
    for i in range(len(ses.acquisitions())):
        acq = ses.acquisitions()[i]
        filelist = [x for x in acq.files if x.type in ["nifti"]]
        for f in filelist:
            #print("searching for {}_.+x{} in {}".format(ses.subject.label,ses.label, f.name))
            search = re.search(r"{}_.+x{}".format(ses.subject.label,ses.label), f.name)
            if search is not None:
                asl_index = i
                is_asl = True
            else:
                continue
        if is_asl:
            return asl_index

## The Procedure to loop

In [42]:
q = identify_recon(sessions[177])

if q != None:
    sessions[177].acquisitions()[recon_index(sessions[177])].download_file(q.name, "asl_m0_combined.nii.gz")
else:
    print("not found")


#if q != None:
#    sessions[177].acquisitions()[12].download_file(q.name, "asl_m0_combined.nii.gz")
#else:
#    print("not found")

searching for 88608_.+x10165 in B0map_9_ph.nii.gz
searching for 88608_.+x10165 in B0map_8_e2.nii.gz
searching for 88608_.+x10165 in B0map_8_e1.nii.gz
searching for 88608_.+x10165 in TOF_3D_multi-slab_7.nii.gz
searching for 88608_.+x10165 in TOF_3D_multi-slab_6.nii.gz
searching for 88608_.+x10165 in TOF_3D_multi-slab_5.nii.gz
searching for 88608_.+x10165 in TOF_3D_multi-slab_4.nii.gz
searching for 88608_.+x10165 in T2_sagittal_SPACE_3.nii.gz
searching for 88608_.+x10165 in MPRAGE_TI1100_ipat2_2.nii.gz
searching for 88608_.+x10165 in bbl1_fracback1_231mb_16.nii.gz
searching for 88608_.+x10165 in DTI_MultiShell_topup_ref_15.nii.gz
searching for 88608_.+x10165 in DTI_MultiShell_117dir_13.nii.gz
searching for 88608_.+x10165 in 88608_20160526x10165.nii.gz
searching for 88608_.+x10165 in ASL_dicomref_12.nii.gz
searching for 88608_.+x10165 in B0map_9_ph.nii.gz
searching for 88608_.+x10165 in B0map_8_e2.nii.gz
searching for 88608_.+x10165 in B0map_8_e1.nii.gz
searching for 88608_.+x10165 in TOF

In [20]:
file=nb.load('asl_m0_combined.nii.gz')

In [21]:
file

In [22]:
data1=file.get_data()
data1.shape

(64, 64, 32, 82)

In [23]:
asl=data1[:,:,:,0:80]
m0=data1[:,:,:,80:82]

In [24]:
aslfile=nb.Nifti1Image(dataobj=asl,header=file.header,affine=file.affine)
mofile=nb.Nifti1Image(dataobj=m0,header=file.header,affine=file.affine)

In [25]:
nb.save(aslfile,filename='asl_reconstructed.nii.gz')
nb.save(mofile,filename='m0_reconstructed.nii.gz')

In [ ]:
# Upload the split M0 and ASL
acquisition = session.add_acquisition(label='ASL')

acquisition.upload_file('asl_reconstructed.nii.gz')
acquisition.upload_file('m0_reconstructed.nii.gz')

In [26]:
# Cleanup
os.system('rm asl_reconstructed.nii.gz')
os.system('rm m0_reconstructed.nii.gz')
os.system('rm asl_m0_combined.nii.gz')

0

### The Loop 

In [106]:
for i in range(len(sessions)):

    # check if it's in the list of IDs 
    if (sessions[i].label in recon_id.values) and (sessions[i].label not in done_id.values):
        
        # download the combined nifti
        q = identify_recon(sessions[i])

        if q != None:
            sessions[i].acquisitions()[recon_index(sessions[i])].download_file(q.name, "asl_m0_combined.nii.gz")
            
            # load in the combined nifti
            if os.path.getsize('/Users/diegodavila/Documents/ASL_TEMPORARY_STORAGE/asl_m0_combined.nii.gz') > 0:
                file=nb.load('asl_m0_combined.nii.gz')
            else:
                os.system('rm asl_m0_combined.nii.gz')
                

            # split procedure
            data1=file.get_data()
            asl=data1[:,:,:,0:80]
            m0=data1[:,:,:,80:82]
            aslfile=nb.Nifti1Image(dataobj=asl,header=file.header,affine=file.affine)
            mofile=nb.Nifti1Image(dataobj=m0,header=file.header,affine=file.affine)
            nb.save(aslfile,filename='asl_reconstructed.nii.gz')
            nb.save(mofile,filename='m0_reconstructed.nii.gz')

            # Upload to Flywheel
            acquisition = sessions[i].add_acquisition(label='ASL')
            acquisition.upload_file('asl_reconstructed.nii.gz')
            acquisition.upload_file('m0_reconstructed.nii.gz')
            print("Uploading ASL and M0 to {}".format(sessions[i].label))

            # Cleanup
            os.system('rm asl_reconstructed.nii.gz')
            os.system('rm m0_reconstructed.nii.gz')
            os.system('rm asl_m0_combined.nii.gz')


        else:
            print("Reconstructed ASL was not found in {}".format(sessions[i].label))


Reconstructed ASL was not found in 9942
Uploading ASL and M0 to 9995
Reconstructed ASL was not found in 9970
Reconstructed ASL was not found in 9969
Reconstructed ASL was not found in 9968
Reconstructed ASL was not found in 9958
Reconstructed ASL was not found in 9956
Reconstructed ASL was not found in 9955
Reconstructed ASL was not found in 9947
Uploading ASL and M0 to 10180
Uploading ASL and M0 to 10165
Uploading ASL and M0 to 10163
Uploading ASL and M0 to 10161
Uploading ASL and M0 to 10152
Uploading ASL and M0 to 10151
Uploading ASL and M0 to 10150
Uploading ASL and M0 to 10149
Uploading ASL and M0 to 10148
Uploading ASL and M0 to 10147
Uploading ASL and M0 to 10133
Uploading ASL and M0 to 10132
Uploading ASL and M0 to 10131
Reconstructed ASL was not found in 10134
Reconstructed ASL was not found in 10127
Reconstructed ASL was not found in 10156
Reconstructed ASL was not found in 10158
Reconstructed ASL was not found in 10115
Reconstructed ASL was not found in 10153
Reconstructed A

### Which are missing the recon ASL?

In [107]:
for i in range(len(sessions)):
    if (sessions[i].label in recon_id.values):     
        q = identify_recon(sessions[i])
        
        if q != None:
            continue 
        else:
            print("{}".format(sessions[i].label))



9942
9970
9969
9968
9958
9956
9955
9947
10134
10127
10156
10158
10115
10153
10136
10137
10117
10145
10138
10121
10120
10126
10142
